In [ ]:
import base64
import datetime
import os
import re
from typing import TypedDict

import markdown2
import markdownify
import pandas as pd
import tiktoken
from bs4 import BeautifulSoup
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from LLM_get_folder import get_local_folder
from pydantic import BaseModel, Field
from utilities.wordpress_tools import (
    check_insert_image,
    get_rewrite_post_ID,
    insert_keyword_url,
    post_wordpress_post,
    retrieve_wordpress_post,
    tags_to_IDs,
    update_summary_qa,
)

gpt_model_small = "gpt-4o-mini"
gpt_model_name = "gpt-4o-mini"
llm_small = ChatOpenAI(model=gpt_model_small, timeout=120, temperature=0)
llm = ChatOpenAI(model=gpt_model_name, timeout=120, temperature=0)

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "forwardpathway_post_rewrite"
wp_url = "https://www.forwardpathway.com/wp-json/wp/v2"
wp_post_url = wp_url + "/posts"
wp_media_url = wp_url + "/media"
wp_tag_url = wp_url + "/tags"

user_id = os.environ["wordpress_username"]
# user app password can be created in the user/edit user/application password
user_app_password = os.environ["wordpress_pass"]

credentials = user_id + ":" + user_app_password
token = base64.b64encode(credentials.encode())
header = {"Authorization": "Basic " + token.decode("utf-8")}

In [12]:
class GraphState(TypedDict):
    post_ID: int
    URL: str
    post_title: str
    raw_html: str
    original_content: str
    text_content: str
    revised_content: str
    revises: list
    tags: set
    feature_image_ID: int

In [13]:
def get_post_content(state):
    post_ID = state["post_ID"]
    URL = "https://www.forwardpathway.com/" + str(post_ID)
    response = retrieve_wordpress_post(post_ID)
    post_title = response.json()["title"]["rendered"]
    raw_html = response.json()["content"]["rendered"]
    feature_image_ID = response.json()["featured_media"]
    tags = response.json()["tags"]
    soup_content = BeautifulSoup(raw_html, "html.parser")
    imgs = soup_content.find_all("img")
    remove_attrs = set(["srcset", "class", "decoding", "height", "sizes", "width"])
    for img in imgs:
        img_attrs = dict(img.attrs)
        for img_attr in img_attrs:
            if img_attr in remove_attrs:
                del img.attrs[img_attr]
    if soup_content is not None:
        elements = soup_content.find_all(
            True,
            class_=[
                "crp_related",
                "topBanner",
                "bottomBanner",
                "wp-block-advgb-summary",
                "yoast-table-of-contents",
                "exclusiveStatement",
                "companyLocation",
                "CommentsAndShare",
                "AI_Summary",
                "AI_QA",
                "btn-group",
                "current_usnews_ranking",
            ],
        )
        for element in elements:
            element.decompose()  # delete elements in class array
        elements = soup_content.find_all(
            True,
            id=[
                "crp_related",
            ],
        )
        for element in elements:
            element.decompose()  # delete elements in id array
        elements = soup_content.findAll(["svg", "style", "script", "noscript"])
        for element in elements:
            element.decompose()  # delete elements of other types like scripts and style
        others = soup_content.find_all(["h1", "h2", "h3", "h4", "h5"])
        # for others_element in others:
        #    if (
        #        "class" in others_element.attrs
        #        and "wp-block-heading" in others_element.get("class")
        #    ):
        #        if len(others_element.get("class")) > 1:
        #            others_element.attrs["class"].remove("wp-block-heading")
        #        else:
        #            del others_element.attrs["class"]
        #        if "id" in others_element.attrs:
        #            del others_element.attrs[
        #                "id"
        #            ]  # delete headings classes/id with wp-block-heading

        amcharts = soup_content.find_all("div", id=re.compile("^amcharts-"))
        for amchart_element in amcharts:  # replace amcharts with amcharts shortcode
            amchart_id = amchart_element.get("id").replace("amcharts-", "")
            amchart_element.replace_with(
                """[amcharts id="chart-{}"]""".format(amchart_id)
            )

        videos = soup_content.find_all("figure", class_="wp-block-video")
        for video in videos:
            video_poster = video.video.attrs["poster"]
            video_src = video.video.attrs["src"]
            video.replace_with(
                """[video poster="{}" src="{}"]""".format(video_poster, video_src)
            )
    # original_content = str(soup_content.find_all(True))
    original_content = markdownify.markdownify(str(soup_content), escape_misc=False)
    return {
        "post_title": post_title,
        "raw_html": raw_html,
        "original_content": original_content,
        "URL": URL,
        "tags": tags,
        "feature_image_ID": feature_image_ID,
    }

In [6]:
class revise_single(BaseModel):
    comment: str = Field(description="文章内容具体修改意见")
    search_query: str = Field(
        description="文章内容修改所需资料的具体英文搜索词条，可以用该词条在Google上搜索所需的资料来修改文章"
    )


class revise_output(BaseModel):
    revises: list[revise_single] = Field(
        description="包含文章内容修改意见和具体搜索词条的数组"
    )


def get_revise_comments(state):
    post_title = state["post_title"]
    post_content = state["original_content"]
    revise_comment_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """今天的日期是{today}，下面将给出一篇网站文章内容和该文章标题，请对该文章的内容且只对内容提出具体的修改意见和搜索词条。修改意见可以包含但不限于以下几个方面：\
                使用数据过时、前后逻辑不清晰、用词是否恰当、前后文是否对应。
                修改意见要求：
                1. 内容扩展建议：如果文章长度少于3000个中文字，请提供增加文章长度的建议，并详细描述应增加的内容及搜索该内容的具体搜索词条。
                2. 数据和信息的准确性：检查文章中的数据和信息是否准确和最新。如果发现数据或信息过时，请提出具体的修改建议及搜索更新信息的搜索词条，该搜索词条需要包含该信息的具体内容以便搜索时间。
                3. 信息具体时间确实：检查文章中事件或信息的具体时间，如果有缺失或不准确的，请给出具体的修改建议及搜索该事件或信息具体时间的搜索词条。比如时间缺少年份的请给出补充时间年份的建议，\
                同时搜索词条应该给出能搜索到该事件具体时间的搜索词条。注意：事件的时间不要默认为今年或今天，应该给出具体搜索词条去搜索该事件的时间。
                4. 逻辑和结构：评估文章的逻辑和结构，确保前后文连贯，文章内容契合文章标题和主题。如果发现逻辑不清晰或结构混乱的地方，请指出并给出修改建议及搜索相关信息的搜索词条。
                5. 用词和语言：检查文章的用词是否恰当，语言是否流畅。如果发现用词不当或语言不流畅的地方，请提出修改建议及搜索相关信息的搜索词条。
                6. 引用和出处：检查文章中的引用和出处是否清晰明确。如果发现引用不清楚或缺乏出处的地方，请提出修改建议及搜索相关信息的搜索词条。
                7. 修改意见要具体：请具体指出文章中哪些部分需要修改，详细描述不好的地方，如数据过时、前后逻辑不清晰、用词不当等并对应的给出搜索相关信息的搜索词条。
                搜索词条要求：修改意见与搜索词条需一一对应。每一条修改意见应对应一个具体的搜索词条。搜索词条必须是英文，且应能在Google上搜索到相应的资料。
                输出格式：请给出3条修改意见，并将每一条修改意见与搜索词条放入对应的输出数组中。确保每条修改意见具体明确，并对应相关的搜索词条。""",
            ),
            ("human", "文章标题:{title}\n\n文章内容: {content}"),
        ]
    )
    today = datetime.datetime.now().strftime("%Y年%m月%d日")
    revise_comment_llm = llm.with_structured_output(revise_output, method="json_schema")
    revise_comment_chain = revise_comment_prompt | revise_comment_llm
    response = revise_comment_chain.invoke(
        {"today": today, "title": post_title, "content": post_content}
    )
    return {"revises": response.revises}

In [7]:
def revise_post(state):
    concat_max = 3
    MAX_DOC_PER_SEARCH = 4
    MAX_TOKEN_LENGTH = 100000
    post_title = state["post_title"]
    revises = state["revises"]
    revise_content = state["original_content"]
    revise_n = 0
    system_common = """任务说明：你是一名专业的网站内容编辑。下面将给出一篇网站文章内容和该文章标题，并提供一条修改意见以及与该修改意见相关的一些参考资料。\
    请根据修改意见和参考资料对原文章内容进行修改，尽量保持文章长度不减少。最终输出结果必须是markdown格式输出，但是不要在文章最前端添加markdown标志。今天的日期是{today}
    修改要求：
    1. 内容更新：请参考修改意见和相关资料对原文进行修改。如果更新内容涉及表格数据，请对整个表格进行更新，且更新内容必须来自参考资料。如果参考资料中不包含关键信息，请删除原表格中的相关内容。\
    修改后的文章内容要尽量贴合文章标题和原文章主题。在文章的最开始位置添加这次更新的更新时间和更新内容简短概述，更新记录最多保留不同日期的3条记录，这段更新记录必须在文章的最开始位置。
    2. 信息具体时间更新：如果更新内容涉及具体时间的，修改年份和时间必须从参考资料中获取且必须要准确无误，如果参考资料中不包含所需要的具体时间则不更新该内容时间。
    3. 排版要求：修改后的文章需输出修改后的全文内容，且仅输出文章内容，不要包含文章标题或者其他内容。对遇到的表格进行表格排版，各级标题统一格式排版。
    4. 中文名翻译：如果原文中提到的美国大学只有英文名，请添加中文名翻译。
    5. 图片链接处理：如果原文中有图片链接请保留原文中的图片链接，不要删除该图片链接，也不要改动图片链接，请保留原文中的所有图片链接。
    6. 数据图代码处理：如果原文中有类似[amcharts id="chart-xx"]的代码，"这些代码请保留原有代码，不可以有任何改动"。"""
    system_diff = [
        """\n7. 参考标记和编号：文章原文末尾如果有参考资料部分的，请先删除文章末尾原有的参考资料，同时删除原文中所有参考资料编号，参考资料编号也可能是"【1】"，"【2】"，"【3】"等。\
        然后如果修改是参考了搜索资料，请在文章内容中添加参考标记与编号，并在文章末尾添加参考过的资料与编号，文章内容中的参考资料编号需要与文章末尾的编号一一对应。如果没做修改或者修改没有参考的搜索资料不要添加。\
        参考资料部分需统一排版并编号，参考资料如果有重复请删除重复的资料，如果文章末尾参考资料编号做了改动，与之对应的文章内容中的参考资料标号也要做修改。""",
        """\n7. 参考标记和编号：如果修改是参考了搜索资料，请在文章内容中添加参考标记与编号，并在文章末尾添加参考过的资料与编号，文章内容中的参考资料编号需要与文章末尾的编号一一对应。如果没做修改或者修改没有参考的搜索资料不要添加。\
        参考资料部分需统一排版并编号，参考资料如果有重复请删除重复的资料，如果文章末尾参考资料编号做了改动，与之对应的文章内容中的参考资料标号也要做修改。""",
    ]
    today = datetime.datetime.now().strftime("%Y年%m月%d日")
    for revise in revises:
        original_content = revise_content
        revise_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_common + system_diff[revise_n >= 1]),
                (
                    "human",
                    "文章标题:{title}\n\n原文章内容: {content}\n\n修改意见:{comment}\n\n参考资料:{docs}",
                ),
            ]
        )
        revise_chain = revise_prompt | llm
        comment = revise.comment
        search_query = revise.search_query
        search = GoogleSerperAPIWrapper()
        results = search.results(
            search_query + " -filetype:pdf -site:forwardpathway.com"
        )
        links = []
        docs = []
        print("--------get search results done!---------------")
        search_result_count = 0
        total_token = 0
        for x in results["organic"]:
            if search_result_count >= MAX_DOC_PER_SEARCH:
                break
            try:
                loader = WebBaseLoader(
                    x["link"], requests_kwargs={"timeout": 15}, continue_on_failure=True
                )
                new_doc = loader.load()
            except:
                continue
            encoding = tiktoken.encoding_for_model(gpt_model_name)
            token_length = len(encoding.encode(str(new_doc[0])))
            if (total_token + token_length) >= MAX_TOKEN_LENGTH:
                break
            if token_length <= 50000:
                docs = docs + new_doc
                search_result_count += 1
                total_token = total_token + token_length

        print("--------load docs done!---------------")
        response = revise_chain.invoke(
            {
                "today": today,
                "title": post_title,
                "content": revise_content,
                "comment": comment,
                "docs": docs,
            }
        )
        revise_content = response.content
        stop_reason = response.response_metadata["finish_reason"]
        print("----stop reason:", stop_reason, "----")
        concat_count = 0
        while stop_reason == "length":
            print("---continue generate next part---")
            if concat_count >= concat_max:
                raise Exception("max count reached")
            revise_prompt = ChatPromptTemplate.from_messages(
                [
                    (
                        "system",
                        "你已经生成了第一部分输出内容，请根据下面相同的任务需求及第一部分输出内容继续生成剩余的内容。\n\n"
                        + system_common
                        + system_diff[revise_n >= 1],
                    ),
                    (
                        "human",
                        "文章标题:{title}\n\n原文章内容: {content}\n\n修改意见:{comment}\n\n参考资料:{docs}\n\n第一部分输出内容：{first_part_content}",
                    ),
                ]
            )
            revise_chain = revise_prompt | llm
            response = revise_chain.invoke(
                {
                    "today": today,
                    "title": post_title,
                    "content": original_content,
                    "comment": comment,
                    "docs": docs,
                    "first_part_content": revise_content,
                }
            )
            revise_content = revise_content + response.content
            stop_reason = response.response_metadata["finish_reason"]
            concat_count += 1
        revise_n += 1
        print("-----------{}x revise done!---------------".format(revise_n))
    return {"revised_content": revise_content}

In [8]:
def replace_amcharts_code(content):
    amcharts = re.finditer(r"""\[amcharts id="chart\-(\d+)"\]""", content)
    for amchart in amcharts:
        amchart_id = amchart.group(1)
        content = content.replace(
            """[amcharts id="chart-{}"]""".format(amchart_id),
            """<div id="amcharts-{}">[amcharts id="chart-{}"]</div>""".format(
                amchart_id, amchart_id
            ),
        )
    return content

In [9]:
def replace_videos_code(content):
    videos = re.finditer(r"""\[video poster="(.*)" src="(.*)"\]""", content)
    for video in videos:
        video_poster = video.group(1)
        video_src = video.group(2)
        content = content.replace(
            """[video poster="{}" src="{}"]""".format(video_poster, video_src),
            """<figure class="wp-block-video"><video controls="" poster="{}" preload="none" src="{}"></video></figure>""".format(
                video_poster, video_src
            ),
        )
    return content

In [10]:
class post_title_tags(BaseModel):
    title: str = Field(description="根据文章内容更新后的新标题")
    tags: list[str] = Field(description="根据文章内容生成的标签")


def update_post(state):
    post_ID = state["post_ID"]
    post_title = state["post_title"]
    raw_content = state["revised_content"]
    tags = state["tags"]
    feature_image_ID = state["feature_image_ID"]
    title_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """你的角色是一名网站编辑，用户给你内一篇关于美国留学的文章内容与这篇文章的老标题，请根据文章内容更新老标题，标题风格与内容需要与老标题相似，只根据提供的文章内容对老标题做出必要的修改。\
                修改文章标题中的时间需要谨慎，忽略文章最开始的更新时间和更新记录，尽量根据文章主体内容来修改标题。
                最后输出内容只包含更新后的标题，不要包含其他内容，比如不要包含'更新后的标题'这类内容，也不要包含老标题，只输出更新后的新标题。
                同时根据文章内容及现有标题生成一些与文章内容相关的标签。""",
            ),
            ("human", "文章老标题:{title}\n\n文章内容: {content}"),
        ]
    )
    title_chain = title_prompt | llm_small.with_structured_output(
        post_title_tags, method="json_schema"
    )
    try:
        response = title_chain.invoke({"title": post_title, "content": raw_content})
        new_title = response.title
        generated_tags = tags_to_IDs(response.tags)
    except:
        new_title = post_title
        generated_tags = set()
    content = markdown2.markdown(raw_content, extras=["tables", "footnotes"])
    content = replace_amcharts_code(content)
    content = replace_videos_code(content)
    content, link_tags = insert_keyword_url(content)
    content = check_insert_image(content, feature_image_ID)
    tags = set(tags) | generated_tags | link_tags
    post_wordpress_post(
        post_ID=post_ID, post_title=new_title, post_body=content, tags=tags
    )
    update_summary_qa(post_ID, raw_content, llm_small)
    folder = get_local_folder()
    filepath = os.path.join(folder, "post_rewrite.csv")
    df = pd.DataFrame(
        [[post_ID, post_title, new_title, state["raw_html"], content]],
        columns=["post_ID", "post_title", "new_title", "raw_html", "revised"],
    )
    df.to_csv(filepath, mode="a", index=False, header=False)
    return

In [14]:
######################## Build LangGraph ####################################
workflow = StateGraph(GraphState)
workflow.add_node("get_post_content", get_post_content)
workflow.add_node("get_revise_comments", get_revise_comments)
workflow.add_node("revise_post", revise_post)
workflow.add_node("update_post", update_post)

workflow.set_entry_point("get_post_content")
workflow.add_edge("get_post_content", "get_revise_comments")
workflow.add_edge("get_revise_comments", "revise_post")
workflow.add_edge("revise_post", "update_post")
workflow.add_edge("update_post", END)

app = workflow.compile()

In [ ]:
from IPython.display import Image as IPImage
from IPython.display import display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    IPImage(
        app.get_graph().draw_mermaid_png(
            curve_style=CurveStyle.BASIS,
            node_colors=NodeStyles(
                first="fill:#FDFFB6",
                last="fill:#FFADAD",
                default="fill:#CAFFBF,line-height:1",
            ),
            draw_method=MermaidDrawMethod.API,
        ),
        width=300,
    )
)

img = app.get_graph().draw_mermaid_png(
    curve_style=CurveStyle.BASIS,
    node_colors=NodeStyles(
        first="fill:#FDFFB6",
        last="fill:#FFADAD",
        default="fill:#CAFFBF,line-height:1",
    ),
    draw_method=MermaidDrawMethod.API,
)
with open("post_rewrite_flow.png", "wb") as png:
    png.write(img)

In [16]:
app.invoke({"post_ID": get_rewrite_post_ID()})
#app.invoke({"post_ID": 40525})

--------get search results done!---------------
--------load docs done!---------------
----stop reason: stop ----
-----------1x revise done!---------------
--------get search results done!---------------
--------load docs done!---------------
----stop reason: stop ----
-----------2x revise done!---------------
--------get search results done!---------------
--------load docs done!---------------
----stop reason: stop ----
-----------3x revise done!---------------


{'post_ID': 82087,
 'URL': 'https://www.forwardpathway.com/82087',
 'post_title': '春季开学，美国大学会强制要求学生接种新冠疫苗吗？',
 'raw_html': '\n<p>很快就要开学了，春季学期如何有效地控制校园疫情依然是首要问题。目前，基本所有的大学都提供免费的核酸检测，希望学生能在返校前和开学初完成新冠病毒的检测以初步控制病毒的传播。瑞辉疫苗12月初在美国获批，一时间各地都在争相报道疫苗接种的盛况。很多大型企业都表示会在复工前要求员工接种疫苗。那么，为了控制疫情，学校会强制要求学生接种新冠疫苗吗？</p>\n\n\n\n<h2 class="wp-block-heading" id="h-">学校要求接种疫苗并非史无前例</h2>\n\n\n\n<p>大家一开始听到学校强制要求疫苗还会觉得陌生，其实仔细想想，美国的大学本来就有强制要求疫苗接种的惯例。很多年以来，美国的大学为了减少校园疾病传播都对入学的学生有疫苗接种要求。一般强制要求接种的有针对脑膜炎，麻疹和水痘等疾病的疫苗。如果想要申请Waive疫苗要求，可以提交医疗、宗教等原因的申请。而且学校的态度也是非常鲜明的，比如<a href="https://www.forwardpathway.com/9686">宾夕法尼亚州立大学</a>的官网就明确说了“For the safety of our campus community, students who do not supply proper evidence of immunity may be removed from campus during a communicable disease outbreak.”译：为了我们校园社区的安全，在传染病暴发期间，未提供适当免疫力证据的学生可能会被赶出校园。</p>\n\n\n\n<p>在疫情爆发如此严重的情况下，如果美国的学校要求学生接种疫苗也是合情合理的。其实有一些学校已经采取行动了。比如，Ursinus College，这是一所小型的私立文理学院。学校已经开始申请获得新冠疫苗。学校也是美国众多的疫苗“封闭分发”地点之一。这就意味着学校有疫苗分发和注射的工作人员和设施。全美范围内有多所学校的医学院都是疫苗分发地。</p